# Author: Justin Hsi
## Part 2 of cleaning lending club payment history

In [1]:
import dir_constants as dc
from tqdm import tqdm_notebook, tqdm

project = 'lendingclub'

In [2]:
def find_dupe_dates(group):
    return pd.to_datetime(group[group.duplicated('date')]['date'].values)

def merge_dupe_dates(group):
    df_chunks = []
    
    dupe_dates = find_dupe_dates(group)
    df_chunks.append(group[~group['date'].isin(dupe_dates)])
    
    for date in dupe_dates:
        problem_rows = group[group['date'] == date]
        ori_index = problem_rows.index
        keep_row = problem_rows.iloc[-1].to_dict()
        keep_row['outs_princp_beg'] = problem_rows.ix[ori_index[0],column_iloc_map['outs_princp_beg']]
        
        summed = problem_rows.sum()
        keep_row['princp_paid'] = summed['princp_paid']
        keep_row['int_paid'] = summed['int_paid']
        keep_row['fee_paid'] = summed['fee_paid']
        keep_row['amt_due'] = summed['amt_due']
        keep_row['amt_paid'] = summed['amt_paid']
        keep_row['charged_off_amt'] = summed['charged_off_amt']
        keep_row['recovs'] = summed['recovs']
        keep_row['recov_fees'] = summed['recov_fees']
            
        to_append = pd.DataFrame(keep_row, index=[ori_index[-1]])
        df_chunks.append(to_append)
    return pd.concat(df_chunks)

In [3]:
# !ls {dc.data_path+project}
path = dc.data_path+project

pmt_hist = pd.read_feather(path+'/pmt_hist_c1.fth')

# There are loans that have multiple row entries per month (as in multiple pmts in same month) and there are also loans that don't have any entry for a month

In [4]:
column_iloc_map = {
    col_name: pmt_hist.iloc[-1].index.get_loc(col_name)
    for col_name in pmt_hist.columns.values
}

In [5]:
dup_date_ids = pmt_hist[pmt_hist.duplicated(
    ['loan_id', 'date'])]['loan_id'].unique()
already_good = pmt_hist[~pmt_hist['loan_id'].isin(dup_date_ids)]
needs_fixing = pmt_hist[pmt_hist['loan_id'].isin(dup_date_ids)]
del pmt_hist
gc.collect()

395

In [6]:
fixed_dfs = []
id_grouped = needs_fixing.groupby('loan_id')
for ids, group in tqdm(id_grouped):
    if ids in dup_date_ids:
        fixed_dfs.append(merge_dupe_dates(group))

  0%|          | 0/10503 [00:00<?, ?it/s]/home/justin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
100%|██████████| 10503/10503 [02:54<00:00, 60.14it/s]


In [9]:
fixed_df = pd.concat(fixed_dfs)

In [10]:
pmt_hist = pd.concat([already_good, fixed_df])
del already_good, fixed_df
gc.collect()

0

In [11]:
pmt_hist.shape

(37316502, 41)

# store before next cleaning step

In [12]:
pmt_hist.reset_index(inplace=True, drop=True)

In [13]:
pmt_hist.to_feather(path+'/pmt_hist_c2.fth')

# old stuff

In [ ]:
# old version for chunking through hdfs
# pmt_hist_ids = store['pmt_hist_ids'].astype(int)
# max_id = pmt_hist_ids.max()
# chunksize = 800
# n_chunks = len(pmt_hist_ids)//chunksize + 1

# # fix loans with double month entries _________________________________________
# # left_bound = 0
# # right_bound = pmt_hist_ids[chunksize]
# already_good_dfs = []
# fixed_dfs = []
# k = 0
# for n in tqdm_notebook(np.arange(n_chunks)):
#     if n == 0:
#         left_bound = 0
#     else:
#         left_bound = pmt_hist_ids[n*chunksize]
#     if n == (n_chunks - 1):
#         right_bound = max_id
#     else:
#         right_bound = pmt_hist_ids[(n+1)*chunksize]
    
#     chunk = pd.read_hdf(
#         store,
#         'pmt_hist_intermediary_1',
#         where='(loan_id_num > left_bound) & (loan_id_num <= right_bound)')
#     loans_with_two_entries_in_same_month = chunk[chunk.duplicated(
#     ['loan_id', 'date'])]
#     dup_date_ids = loans_with_two_entries_in_same_month['loan_id'].unique()
#     if k == 0:
#         column_iloc_map = {
#             col_name: chunk.iloc[-1].index.get_loc(col_name)
#             for col_name in chunk.columns.values
#         }
#         k += 1

#     id_grouped = chunk.groupby('loan_id')
#     already_good = chunk[~chunk['loan_id'].isin(dup_date_ids)]
#     for ids, group in id_grouped:
#         if ids in dup_date_ids:
#             fixed_dfs.append(merge_dupe_dates(group))
#         else:
#             pass

#     already_good_dfs.append(already_good)

In [ ]:
# # Create min_itemsize_dict for allocating size when storing ___________________
# min_itemsize_dict = {}
# for col in already_good.columns:
#     if already_good[col].dtype == np.object:
#         print(col, already_good[col].str.len().max())
#         if col in ['State', 'VINTAGE', 'grade']:
#             pass
#         else:
#             min_itemsize_dict[col] = 15

# col_dtype_map = already_good_dfs[0].dtypes.to_dict()
# all_fixed_dfs = pd.concat(fixed_dfs)
# for col, dtype in col_dtype_map.items():
#     all_fixed_dfs[col] = all_fixed_dfs[col].astype(dtype)
     
# k = 0
# for chunk in tqdm_notebook([all_fixed_dfs] + already_good_dfs):
#     if k == 0:
#         store.append(
#             'pmt_hist_intermediary_2',
#             chunk,
#             data_columns=True,
#             index=True,
#             append=False,
#             min_itemsize=min_itemsize_dict)
#         k += 1
#     else:
#         store.append(
#             'pmt_hist_intermediary_2',
#             chunk,
#             data_columns=True,
#             index=True,
#             append=True)           
        
# store.close()        